In [65]:
# using %matplotlib inline to immediatly draw plot after run
%matplotlib inline 

# imports
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import dotenv
import os
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler, StandardScaler

"""prepare environment variable"""
project_dir = os.path.join(os.path.abspath(''), os.pardir)
dotenv_path = os.path.join(project_dir, '.env')
dotenv.load_dotenv(dotenv_path)

True

In [66]:
# functions
def check_path_existance(path):
    return os.path.exists(path)

In [67]:
dataset_folder = os.path.join(project_dir, os.getenv('RAW_FOLDER'), 'house-prices-advanced-regression-techniques')
train_df = None
test_df = None

if check_path_existance(dataset_folder):   
    train_df = pd.read_csv(os.path.join(dataset_folder, 'train.csv'))
    test_df = pd.read_csv(os.path.join(dataset_folder, 'test.csv'))

# Data Preparation

In [76]:
"""1. DATA CLEANING"""
"""Copy Data for Reuse"""
dataset = train_df.copy()

"""Fill NA Categorical Column on Purpose"""
categorical_columns_nan_on_purpose = [
    'Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 
    'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
    'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 
    'GarageCond', 'PoolQC', 'Fence', 'MiscFeature',
]

def replace_missing_categorical_column_on_purpose(df: pd.DataFrame) -> pd.DataFrame:
    try:
        df = df.copy()
        for f in categorical_columns_nan_on_purpose:
            df[f].fillna('Missing', inplace=True)
        return df
    except AttributeError:
        print("Make sure to input Pandas DataFrame")
        return None

dataset = replace_missing_categorical_column_on_purpose(dataset)

"""Fill NA Electrical Column with Mode"""
dataset['Electrical'].fillna(dataset['Electrical'].mode()[0], inplace=True)

"""Fill NA LotFrontage with Mean"""
dataset['LotFrontage'].fillna(dataset['LotFrontage'].mean(), inplace=True)

"""Fill NA MasVnrArea with 0"""
dataset['MasVnrArea'].fillna(0, inplace=True)

"""Fill NA GarageYrBlt with 0 and convert it to int data type"""
dataset['GarageYrBlt'] = dataset['GarageYrBlt'].fillna(-1).astype('int64')

"""Remove Id column"""
dataset.drop(columns=['Id'], inplace=True)

"""Convert MSSubClass and MoSold into categorical column"""
dataset.loc[:, ('MSSubClass', 'MoSold')] = dataset.loc[:, ('MSSubClass', 'MoSold')].astype(str)

"""Perform Log Transformation"""
log_transformed_column = ('LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', '2ndFlrSF', 'LowQualFinSF', 'SalePrice')
dataset.loc[:, log_transformed_column] = dataset.loc[:, log_transformed_column].apply(lambda x: np.log(x+1))
# convert it back using np.exp(x)-1

"""Perform Numerical Feature Selection"""
selected_numerical_columns = [
    'LotFrontage', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 
    'MasVnrArea', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', 
    '2ndFlrSF', 'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 
    'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 
    'SalePrice'
]
dropped_numerical_columns = [c for c in dataset.select_dtypes(include=['int64', 'float64']).columns if c not in selected_numerical_columns]
dataset.drop(columns=dropped_numerical_columns, inplace=True)

"""Perform Numerical Feature Scaling"""
selected_numerical_columns = [
    'LotFrontage', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 
    'MasVnrArea', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', 
    '2ndFlrSF', 'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 
    'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
    'SalePrice'
]
scaler_objects = {}
for c in selected_numerical_columns:
    scaler = MinMaxScaler()
    dataset.loc[:, c] = scaler.fit_transform(dataset.loc[:, [c]])
    scaler_objects[c] = scaler


"""Perform Ordinal Encoding"""
ordinal_categorical_columns = {
    "ExterQual": ["Ex", "Gd", "TA", "Fa", "Po"], 
    "ExterCond": ["Ex", "Gd", "TA", "Fa", "Po"], 
    "BsmtQual":  ["Ex", "Gd", "TA", "Fa", "Po", "Missing"],
    "BsmtCond": ["Ex", "Gd", "TA", "Fa", "Po", "Missing"],
    "BsmtExposure": ["Gd", "Av", "Mn", "No", "Missing"],
    "BsmtFinType1": ["GLQ", "ALQ", "BLQ", "Rec", "LwQ", "Unf", "Missing"],
    "BsmtFinType2": ["GLQ", "ALQ", "BLQ", "Rec", "LwQ", "Unf", "Missing"],
    "HeatingQC": ["Ex", "Gd", "TA", "Fa", "Po"], 
    "KitchenQual": ["Ex", "Gd", "TA", "Fa", "Po"], 
    "FireplaceQu": ["Ex", "Gd", "TA", "Fa", "Po", "Missing"],
    "GarageFinish": ["Fin", "RFn", "Unf", "Missing"], 
    "GarageQual": ["Ex", "Gd", "TA", "Fa", "Po", "Missing"], 
    "GarageCond": ["Ex", "Gd", "TA", "Fa", "Po", "Missing"], 
    "PoolQC": ["Ex", "Gd", "TA", "Fa", "Missing"], 
    "Fence": ["GdPrv", "MnPrv", "GdWo", "MnWw", "Missing"]
} # gather ordinal categorical column
for k, v in ordinal_categorical_columns.items():
    ordinal_encoder = OrdinalEncoder(categories=[v])
    dataset[k] = ordinal_encoder.fit_transform(dataset[[k]]).astype(int)


"""Perform One Hot Encoding"""
ordinal_categorical_columns = {
    "MSSubClass": ["20", "30", "40", "45",	"50", "60", "70", "75", "80", "85", "90", "120", "150", "160", "180", "190"],
    "MSZoning": ["A", "C", "FV", "I", "RH", "RL", "RP", "RM"],
    "Street": ["Pave", "Grvl"],
    "Alley": ["Missing", "Grvl", "Pave"],
    "LotShape": ["Reg", "IR1", "IR2", "IR3"],
    "LandContour": ["Lvl", "Bnk", "Low", "HLS"],
    "Utilities": ["AllPub", "NoSewr", "NoSeWa", "ELO"],
    "LotConfig": ["Inside", "FR2", "Corner", "CulDSac", "FR3"],
    "LandSlope": ["Gtl", "Mod", "Sev"],
    "Neighborhood": ["CollgCr", "Veenker", "Crawfor", "NoRidge", "Mitchel", "Somerst", "NWAmes", "OldTown", "BrkSide", "Sawyer", "NridgHt", "NAmes", "SawyerW", "IDOTRR", "MeadowV", "Edwards", "Timber", "Gilbert", "StoneBr", "ClearCr", "NPkVill", "Blmngtn", "BrDale", "SWISU", "Blueste"],
    "Condition1": ["Norm", "Feedr", "PosN", "Artery", "RRAe", "RRNn", "RRAn", "PosA", "RRNe"],
    "Condition2": ["Norm", "Artery", "RRNn", "Feedr", "PosN", "PosA", "RRAn", "RRAe", "RRNe"],
    "BldgType": ["1Fam", "2fmCon", "Duplex", "TwnhsE", "Twnhs"],
    "HouseStyle": ["2Story", "1Story", "1.5Fin", "1.5Unf", "SFoyer", "SLvl", "2.5Unf", "2.5Fin"],
    "RoofStyle": ["Gable", "Hip", "Gambrel", "Mansard", "Flat", "Shed"],
    "RoofMatl": ["CompShg", "WdShngl", "Metal", "WdShake", "Membran", "Tar&Grv", "Roll", "ClyTile"],
    "Exterior1st": ["AsbShng", "AsphShn", "BrkComm", "BrkFace", "CBlock", "CemntBd", "HdBoard",	"ImStucc", "MetalSd", "Other", "Plywood", "PreCast", "Stone", "Stucco", "VinylSd", "Wd Sdng", "WdShing"],
    "Exterior2nd": ["AsbShng", "AsphShn", "BrkComm", "BrkFace", "CBlock", "CemntBd", "HdBoard",	"ImStucc", "MetalSd", "Other", "Plywood", "PreCast", "Stone", "Stucco", "VinylSd", "Wd Sdng", "WdShing"],
    "MasVnrType": ["BrkFace", "Missing", "Stone", "BrkCmn", "CBlock"],
    "Foundation": ["PConc", "CBlock", "BrkTil", "Wood", "Slab", "Stone"],
    "Heating": ["GasA", "GasW", "Grav", "Wall", "OthW", "Floor"],
    "CentralAir": ["Y", "N"],
    "Electrical": ["SBrkr", "FuseF", "FuseA", "FuseP", "Mix", "Missing"],
    "Functional": ["Typ", "Min1", "Maj1", "Min2", "Mod", "Maj2", "Sev", "Sal"],
    "GarageType": ["Attchd", "Detchd", "BuiltIn", "CarPort", "Missing", "Basment", "2Types"],
    "PavedDrive": ["Y", "N", "P"],
    "MiscFeature": ["Missing", "Shed", "Gar2", "Othr", "TenC", "Elev"],
    "SaleType": ["WD", "New", "COD", "ConLD", "ConLI", "CWD", "ConLw", "Con", "Oth", "VWD"],
    "SaleCondition": ["Normal", "Abnorml", "Partial", "AdjLand", "Alloca", "Family"]
} # one hot encoder column


for k, v in ordinal_categorical_columns.items():
    one_hot_encoder = OneHotEncoder(categories=[v], sparse_output=False, handle_unknown='ignore')
    X_encoded = pd.DataFrame(one_hot_encoder.fit_transform(dataset[[k]]).astype(int))
    X_encoded.columns = one_hot_encoder.get_feature_names_out([k])
    dataset.drop(columns=[k], axis=1, inplace=True)
    dataset = pd.concat([dataset, X_encoded], axis=1)


dataset

,LotFrontage,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,...,SaleType_ConLw,SaleType_Con,SaleType_Oth,SaleType_VWD,SaleCondition_Normal,SaleCondition_Abnorml,SaleCondition_Partial,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family
0,0.413268,0.666667,0.949275,0.883333,0.716038,1,2,1,2,3,...,0,0,0,0,1,0,0,0,0,0
1,0.490307,0.555556,0.753623,0.433333,0.000000,2,2,1,2,0,...,0,0,0,0,1,0,0,0,0,0
2,0.429990,0.666667,0.934783,0.866667,0.690361,1,2,1,2,2,...,0,0,0,0,1,0,0,0,0,0
3,0.383633,0.666667,0.311594,0.333333,0.000000,2,2,2,1,3,...,0,0,0,0,0,1,0,0,0,0
4,0.508439,0.777778,0.927536,0.833333,0.794318,1,2,1,2,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.395769,0.555556,0.920290,0.833333,0.000000,2,2,1,2,3,...,0,0,0,0,1,0,0,0,0,0
1456,0.512839,0.555556,0.768116,0.633333,0.648854,2,2,1,2,3,...,0,0,0,0,1,0,0,0,0,0
1457,0.418925,0.666667,0.500000,0.933333,0.000000,0,1,2,1,3,...,0,0,0,0,1,0,0,0,0,0
1458,0.429990,0.444444,0.565217,0.766667,0.000000,2,2,2,2,2,...,0,0,0,0,1,0,0,0,0,0


1. Drop `Id` column
2. Convert `MSSubClass` and `MoSold` to Categorical Column and leave `OverallQual`, and `OverallCond` because they already represent Ordinal Encoding.
3. Perform Log Transformation to `LotFrontage`, `LotArea`, `MasVnrArea`, `Bsmt  FinSF1`, `BsmtFinSF2`, `BsmtUnfSF`, `2ndFlrSF`, `LowQualFinSF`, and `SalePrice`
4. Select the numerical columns `LotFrontage`, `OverallQual`, `YearBuilt`, `YearRemodAdd`, `MasVnrArea`, `BsmtFinSF1`, `TotalBsmtSF`, `1stFlrSF`, `2ndFlrSF`, `GrLivArea`, `FullBath`, `TotRmsAbvGrd`, `Fireplaces`, `GarageCars`, `GarageArea`, `WoodDeckSF`, and `OpenPorchSF`

# Data Modelling

In [147]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

X, y = dataset.drop(columns=['SalePrice']), dataset.loc[:, ['SalePrice']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)


def undo_scaling_log_transform(results, name):
    return np.exp(scaler_objects[name].inverse_transform(results)) -1

def train(x, y, model):
    return model.fit(x, y)

def pred(x, model):
    return model.predict(x)

model = train(X_train, y_train, LinearRegression())
y_pred = pred(X_test, model)

y_pred = undo_scaling_log_transform(y_pred, 'SalePrice')
y_test = undo_scaling_log_transform(y_test, 'SalePrice')

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('rmse: ', rmse)
print('mae: ', mae)
print('r2: ', r2)

rmse:  35688.47520276665
mae:  18926.41766578931
r2:  0.8339487489977861
